# MTH3302: Méthodes probabilistes et statistiques pour l'I.A.
## Projet

- Vincent Haney
- Félix Pelletier
- Charles-François St-Cyr (275354)
- Mikael Vaillant (2029994)


### Librairies

In [38]:
import Pkg;
packages = ["CSV", "DataFrames", "Distributions", "Gadfly", "MLBase", "Random", "Statistics", "Colors", "GLM", "LinearAlgebra", "Combinatorics", "CategoricalArrays","Distances","NearestNeighbors"];
# peut aps être mis dans une fonction. using doit être top-level
for package in packages
    try
        @eval using $(Symbol(package))
    catch e
        Pkg.add(package)
        @eval using $(Symbol(package))
    end
end
Pkg.update()

include("./library/main.jl")
include("./library/normalize.jl")

    Updating registry at `~/.julia/registries/General.toml`


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


normalize (generic function with 9 methods)

### Chargement des données (partiel pour l'exploration)

In [39]:
data = CSV.read("./data/train.csv", DataFrame)
test = CSV.read("./data/test.csv", DataFrame)

train, valid = partitionData(data, 0.75);

first(train, 5)

Row,ID,cut,color,clarity,depth,table,price,x,y,z
,Int64,String15?,String1,String7,Float64?,Float64,Int64,Float64,Float64?,Float64
1,1,missing,E,SI2,60.2,60.0,5340,6.46,6.5,3.9
2,2,Ideal,G,VVS2,62.7,57.0,1936,5.12,5.05,3.19
3,3,Premium,E,VVS1,62.0,59.0,1012,4.33,4.28,2.67
4,4,Ideal,G,VS2,61.7,57.0,720,4.41,4.38,2.71
5,6,Premium,D,VVS2,61.5,60.0,925,4.36,4.42,2.7


### Visualisation des données

In [40]:
# plot(data, x = :price, Geom.histogram())

In [41]:
println("Format des données (entrainement et validation): ", names(data))

missing_counts = collect(count(ismissing,col) for col in eachcol(data))
println("Nombre de valeurs manquantes dans les différentes colonnes: ", missing_counts)

# zero_counts = [count(x -> x == 0, skipmissing(data[:, col])) for col in names(data)]
zero_counts = hcat([0,0,0,0]',hcat([count(x -> x < 0.05, skipmissing(data[:, col])) for col in [:depth,:table,:price,:x, :y, :z]]...))

println("Nombre de valeurs nulles dans les différentes colonnes: ",zero_counts)

println("Taille de l'ensemble de donnée (entrainement et validation): ",size(data,1))

println("Format des données test: ", names(test))

missing_counts = collect(count(ismissing,col) for col in eachcol(test))
println("Nombre de valeurs manquantes dans les différentes colonnes: ", missing_counts)

zero_counts = hcat([0,0,0,0]',hcat([count(x -> x < 0.05, skipmissing(test[:, col])) for col in [:depth,:table,:x, :y, :z]]...))
println("Nombre de valeurs nulles dans l'ensemble test: ",zero_counts)

println("Taille de l'ensemble de donnée (test): ",size(test,1))



Format des données (entrainement et validation): ["ID", "cut", "color", "clarity", "depth", "table", "price", "x", "y", "z"]
Nombre de valeurs manquantes dans les différentes colonnes: [0, 3806, 0, 0, 2201, 0, 0, 0, 2200, 0]
Nombre de valeurs nulles dans les différentes colonnes: 

[0 0 0 0 13 0 0 4 3 15]
Taille de l'ensemble de donnée (entrainement et validation): 40455
Format des données test: ["ID", "cut", "color", "clarity", "depth", "table", "x", "y", "z"]
Nombre de valeurs manquantes dans les différentes colonnes: [0, 1194, 0, 0, 800, 0, 0, 800, 0]
Nombre de valeurs nulles dans l'ensemble test: [0 0 0 0 5 0 4 4 5]
Taille de l'ensemble de donnée (test): 13485


In [42]:
function fillDataFromAverage(data::DataFrame; is_test::Bool = false)
    avgCut = mode(dropmissing(data, :cut).cut)
    avgColor = mode(dropmissing(data, :color).color)
    avgClarity = mode(dropmissing(data, :clarity).clarity)
    avgDepth = mean(dropmissing(data, :depth).depth)
    avgTable = mean(dropmissing(data, :table).table)
    avgX = mean(dropmissing(data, :x).x)
    avgY = mean(dropmissing(data, :y).y)
    avgZ = mean(dropmissing(data, :z).z)
    # avgVol = mean(dropmissing(data, :volume).volume)

    replace!(data.cut, missing => avgCut)
    replace!(data.color, missing => avgColor)
    replace!(data.clarity, missing => avgClarity)
    replace!(data.depth, missing => avgDepth)
    replace!(data.table, missing => avgTable)
    replace!(data.x, missing => avgX)
    replace!(data.y, missing => avgY)
    replace!(data.z, missing => avgZ)
    # replace!(data.volume, missing => avgVol)

    if (!is_test)
        avgPrice = mean(dropmissing(data, :price).price)
        replace!(data.price, missing => avgPrice)
    end
end

fillDataFromAverage (generic function with 1 method)

In [49]:
train₁ = deepcopy(train)
valid₁ = deepcopy(valid)
valid2 = Matrix(deepcopy(valid))
test₁ = deepcopy(test)

fillDataFromAverage(train₁)
fillDataFromAverage(valid₁)
fillDataFromAverage(test₁, is_test = true)

convertQualitativeToQuantitative(train₁, :color)
# convertQualitativeToQuantitative(train₁, :cut)
convertQualitativeToQuantitative(train₁, :clarity)

convertQualitativeToQuantitative(valid₁, :color)
# convertQualitativeToQuantitative(valid₁, :cut)
convertQualitativeToQuantitative(valid₁, :clarity)

convertQualitativeToQuantitative(test₁, :color)
# convertQualitativeToQuantitative(test₁, :cut)
convertQualitativeToQuantitative(test₁, :clarity)

price_train = select(train₁, (:price))
price_true = select(valid₁, (:price))
test_ids = select(test₁, (:ID))

train₁ = select(train₁, Not(["price","cut","ID","color","clarity","y","depth"]))
valid₁ = select(valid₁, Not(["price","cut","ID","color","clarity","y","depth"]))
test₁ = select(test₁, Not(["cut","ID","color","clarity","y","depth"]))

for col in names(train₁[:,1:5])
    train₁[!, col] = (train₁[!, col] .- mean(train₁[!, col])) ./ std(train₁[!, col])
end

for col in names(valid₁[:,1:5])
    valid₁[!, col] = (valid₁[!, col] .- mean(valid₁[!, col])) ./ std(valid₁[!, col])
end

for col in names(test₁[:,1:5])
    test₁[!, col] = (test₁[!, col] .- mean(test₁[!, col])) ./ std(test₁[!, col])
end



first(train₁,5)

Row,table,x,z,color: E,color: F,color: G,color: H,color: I,color: J,clarity: IF,clarity: SI1,clarity: SI2,clarity: VS1,clarity: VS2,clarity: VVS1,clarity: VVS2
,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.12985,0.649465,0.519764,2.13706,-0.466924,0,0,0,0,0,0,1,0,0,0,0
2,-0.209841,-0.544637,-0.500606,-0.467918,-0.466924,1,0,0,0,0,0,0,0,0,0,1
3,0.683284,-1.24862,-1.24792,2.13706,-0.466924,0,0,0,0,0,0,0,0,0,1,0
4,-0.209841,-1.17733,-1.19043,-0.467918,-0.466924,1,0,0,0,0,0,0,0,1,0,0
5,1.12985,-1.22189,-1.20481,-0.467918,-0.466924,0,0,0,0,0,0,0,0,0,0,1


#### Prédiction

In [44]:

Mat_train = Matrix(train₁)'
k = 4
Mat_valid = Matrix(valid₁)'

kdtree = KDTree(Mat_train)
idxs, dists = knn(kdtree, Mat_valid, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

RMSE = sqrt(mean((prediction .- Matrix(price_true)).^2))


730.4470655783505

In [45]:
# Afin de vérifier que les plus grosses erreurs ne sont pas provoquées par les valeurs manquantes
# On regarde dans le 1/5 des erreurs les plus grande. Si ratio_missing >> 0.2, il y a un problème

err_indivi = sqrt.(((prediction .- Matrix(price_true))[:,1]).^2)
id_err_max = sortperm(err_indivi)[end:-1:1]
data_err_max = valid2[id_err_max[1:div(size(valid2,1),5)],:]

missing_counts1 = collect(count(ismissing,col) for col in eachcol(data_err_max))

missing_counts2 = collect(count(ismissing,col) for col in eachcol(valid2))

ratio_missing = round.([missing_counts1[2],missing_counts1[5],missing_counts1[9]] ./ [missing_counts2[2],missing_counts2[5],missing_counts2[9]],digits=2)

ratio_cut = ratio_missing[1];
ratio_depth = ratio_missing[2];
ratio_y = ratio_missing[3];

println("Le ratio de cut dans les 20% d'erreur les plus grandes: $ratio_cut.")
println("Le ratio de depth dans les 20% d'erreur les plus grandes: $ratio_depth.")
println("Le ratio de y dans les 20% d'erreur les plus grandes: $ratio_y.")

#pour le moment, parfaitement distribué, pcq on considère par les colonnes avec des valeurs manquantes, à vérifier une fois que les changements auront été effectués. 


Le ratio de cut dans les 20% d'erreur les plus grandes: 0.19.
Le ratio de depth dans les 20% d'erreur les plus grandes: 0.2.
Le ratio de y dans les 20% d'erreur les plus grandes: 0.2.


In [46]:
# Afin de vérifier que les plus grosses erreurs ne sont pas provoquées par les valeurs manquantes
# On regarde dans le 1/5 des erreurs les plus grande. Si ratio_missing >> 0.2, il y a un problème

num_zeros = count(x -> x == 0, skipmissing(valid2[observation_croiss[1:div(size(valid2,1),10)],:]))
num_zeros2 = count(x -> x == 0, skipmissing(valid2))

ratio_0 = round(num_zeros/num_zeros2,digits=2)
# Print the result
println("Le ratio de zéro dans les 10% d'erreur les plus grandes: $ratio_0.") #67% des zéros sont dans les dix pourcents les plus élevés, il faudra les remplacer


UndefVarError: UndefVarError: observation_croiss not defined

In [48]:
Mat_train = Matrix(train₁)'
k = 4
Mat_test = Matrix(test₁)'

kdtree = KDTree(Mat_train)
idxs, dists = knn(kdtree, Mat_test, k, true)

prediction = zeros(size(idxs))
for i in 1:size(idxs,1)
    prediction[i] =  mean(Matrix(price_train)[idxs[i]])
end

CSV.write("./results/predictions_knn.csv", DataFrame(ID = test_ids, price = prediction));

ArgumentError: ArgumentError: column name "ID" not found in the data frame; existing most similar names are: "x" and "z"